# Pandas 2

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

- import urllib.request 是用来进行网络请求的
- os.path 是用来处理文件路径的
- zipfile 是用来处理.zip压缩文件的。

In [2]:
import urllib.request
import os.path
import zipfile

In [3]:
data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "../data/babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
407807,CA,M,2021,Wylde,6
369319,CA,M,2008,Maison,10
86206,CA,F,1981,Bertha,77
252013,CA,M,1929,Gaylord,7
378036,CA,M,2011,Gus,10


## query和sort_value按照指定条件查找数据

筛选2020年出生的男宝宝的最频繁的名字

In [4]:
babynames.query('Sex == "M" and Year == 2020').sort_values("Count", ascending = False)

,State,Sex,Year,Name,Count
402534,CA,M,2020,Noah,2631
402535,CA,M,2020,Liam,2431
402536,CA,M,2020,Mateo,2080
402537,CA,M,2020,Sebastian,1996
402538,CA,M,2020,Julian,1688
...,...,...,...,...,...
405091,CA,M,2020,Galen,5
405090,CA,M,2020,Fitzgerald,5
405089,CA,M,2020,Faustino,5
405088,CA,M,2020,Farid,5


In [5]:
babynames.query('Sex == "M" and Year == 2020').sort_values("Name")

,State,Sex,Year,Name,Count
404132,CA,M,2020,Aaden,10
404495,CA,M,2020,Aadi,7
404496,CA,M,2020,Aaditya,7
404976,CA,M,2020,Aalam,5
404244,CA,M,2020,Aamir,9
...,...,...,...,...,...
403973,CA,M,2020,Zyair,13
403237,CA,M,2020,Zyaire,38
404494,CA,M,2020,Zyan,8
405330,CA,M,2020,Zymir,5


名字长短排序

In [6]:
def findLongestName(x):
    return x.str.len()
(
    babynames.query('Sex == "M" and Year == 2020')
             .sort_values("Name", key = lambda x : x.str.len(), ascending = False)
)

,State,Sex,Year,Name,Count
404620,CA,M,2020,Michaelangelo,7
404103,CA,M,2020,Michelangelo,11
404102,CA,M,2020,Maximilliano,11
402582,CA,M,2020,Christopher,763
402636,CA,M,2020,Maximiliano,435
...,...,...,...,...,...
405139,CA,M,2020,Jr,5
404542,CA,M,2020,Cj,7
404824,CA,M,2020,Jj,6
404896,CA,M,2020,Oz,6


## 添加新的列

In [7]:
# 利用str.len()遍历了表格中Name列的数据, 存在name_length这个series中
name_length = babynames["Name"].str.len()

# 创建新列, 值为name_length
babynames["name_lengths"] = name_length

## 修改表格为按照指定关键词排序

In [8]:
babynames = babynames.sort_values(by = "name_lengths", ascending = False)
babynames.head(25)

,State,Sex,Year,Name,Count,name_lengths
341130,CA,M,1997,Ryanchristopher,5,15
348093,CA,M,2000,Franciscojavier,6,15
331124,CA,M,1993,Ryanchristopher,5,15
337819,CA,M,1996,Franciscojavier,8,15
317627,CA,M,1988,Franciscojavier,10,15
320534,CA,M,1989,Franciscojavier,6,15
343125,CA,M,1998,Franciscojavier,6,15
325441,CA,M,1991,Ryanchristopher,7,15
340954,CA,M,1997,Franciscojavier,5,15
325562,CA,M,1991,Franciscojavier,6,15


## 删除某一列, 要指定axis为row还是column

In [9]:
babynames = babynames.drop("name_lengths", axis = "columns")
babynames.head(5)

,State,Sex,Year,Name,Count
341130,CA,M,1997,Ryanchristopher,5
348093,CA,M,2000,Franciscojavier,6
331124,CA,M,1993,Ryanchristopher,5
337819,CA,M,1996,Franciscojavier,8
317627,CA,M,1988,Franciscojavier,10


 ## 想要统计出name中含有dr和ea的次数, 并创建新的列

In [35]:
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)
babynames = babynames.sort_values(by = "dr_ea_count", ascending = False)
babynames.head(5)

,State,Sex,Year,Name,Count,dr_ea_count
108738,CA,F,1988,Deandrea,5,3
131037,CA,F,1994,Leandrea,5,3
115965,CA,F,1990,Deandrea,5,3
101982,CA,F,1986,Deandrea,6,3
311780,CA,M,1985,Deandrea,6,3


## 哪个女性名字的下降率最高 Female Name whose popularity has dropped the most.

In [6]:
max_name = max(babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"])
max_name

6065

In [7]:
current_name = babynames.query("Name == 'Jennifer' and Sex == 'F'")["Count"].iloc[-1]
current_name

88

In [8]:
current_name / max_name

0.014509480626545754

In [9]:
def ratio_to_peak(series):
    peak = max(series)
    current = series.iloc[-1]
    return current / peak

In [10]:
series1 = babynames.query("Name == 'Eva' and Sex == 'F'")["Count"]
ratio_to_peak(series1)

0.5335628227194492

In [11]:
series1

54         29
294        29
540        42
830        71
1177       56
         ... 
224975    466
228648    378
232263    396
235946    343
239661    310
Name: Count, Length: 114, dtype: int64

## 当你想要用上面的方法遍历所有名字找出ratio最低的名字
### Approach 1: loop 
但这不是个好方法, 我们要避免慢速的loop

In [12]:
female_babynames = babynames.query("Sex == 'F'").loc[:,["Name","Count","Year"]] 
female_babynames

,Name,Count,Year
0,Mary,295,1910
1,Helen,239,1910
2,Dorothy,220,1910
3,Margaret,163,1910
4,Frances,134,1910
...,...,...,...
243185,Zeppelin,5,2023
243186,Zhamira,5,2023
243187,Zina,5,2023
243188,Zooey,5,2023


In [81]:
rtps = {}
for name in female_babynames["Name"].unique()[0:10]:
    counts_of_current_name = female_babynames[female_babynames["Name"] == name]["Count"]
    rtps[name] = ratio_to_peak(counts_of_current_name)

rtps = pd.Series(rtps)
rtps

Deandrea      1.000000
Leandrea      1.000000
Andrea        0.737674
Alexandrea    0.716216
Audrea        0.857143
Deandra       0.206897
Leandra       0.275000
Adreana       0.750000
Andreah       1.000000
Adreanna      1.000000
dtype: float64

In [82]:
rtps.sort_values()

Deandra       0.206897
Leandra       0.275000
Alexandrea    0.716216
Andrea        0.737674
Adreana       0.750000
Audrea        0.857143
Deandrea      1.000000
Leandrea      1.000000
Andreah       1.000000
Adreanna      1.000000
dtype: float64

### Approach 2: Groupby and Agg
- groupby 是一个非常强大的功能，它允许你根据一个或多个键（列）对数据进行分组，并应用聚合函数或转换函数。
- agg 方法是 DataFrameGroupBy 对象的一个功能，它允许你对分组后的数据应用一个或多个聚合函数。

![groupby and egg](img/groupby.png)

In [77]:
female_babynames.groupby("Name").agg(ratio_to_peak)

,Count,Year
Name,,
Aadhini,1.000000,1.0
Aadhira,0.500000,1.0
Aadhya,0.760000,1.0
Aadya,0.758621,1.0
Aahana,0.269231,1.0
...,...,...
Zyanya,0.800000,1.0
Zyla,1.000000,1.0
Zylah,1.000000,1.0


- 之所以year都是1, 因为函数默认是找出最大值和最后一个值, 再进行相除
- 按照年份的话, 两者肯定都是相同的, 因此结果为1
- 如果还有其他的列, 比如都是无法进行函数内的运算时, 就不会显示那个列
- 通过下面的显示你可以更好的理解

In [16]:
female_babynames.query('Name == "Aahana"')

,Name,Count,Year
179971,Aahana,6,2007
184205,Aahana,6,2008
188119,Aahana,7,2009
191409,Aahana,12,2010
196324,Aahana,7,2011
199101,Aahana,17,2012
203956,Aahana,9,2013
207655,Aahana,11,2014
216042,Aahana,8,2016
219553,Aahana,10,2017


- 指定具体想要哪一列, 双重括号[["Count"]]则是告诉Pandas我们想要选取的是一个多级索引中的列

In [19]:
rtp_table = female_babynames.groupby("Name")[["Count"]].agg(ratio_to_peak)
rtp_table

,Count
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.760000
Aadya,0.758621
Aahana,0.269231
...,...
Zyanya,0.800000
Zyla,1.000000
Zylah,1.000000


#### 更改列名

In [21]:
rtp_table = rtp_table.rename(columns = {"Count":"Count RTP"})
rtp_table

,Count RTP
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.760000
Aadya,0.758621
Aahana,0.269231
...,...
Zyanya,0.800000
Zyla,1.000000
Zylah,1.000000


In [23]:
rtp_table.sort_values("Count RTP")

,Count RTP
Name,
Debra,0.001512
Debbie,0.002815
Tammy,0.003249
Pamela,0.003602
Cheryl,0.003819
...,...
Kanwal,1.000000
Kansas,1.000000
Kanon,1.000000


## Groupby Puzzles

即使你当前使用的Pandas版本允许你传递函数本身，为了代码的健壮性和未来兼容性，使用字符串形式指定聚合函数是一个好习惯。

### 如果我想要计算这些名字count的总和

In [36]:
puzzle1 = female_babynames.groupby("Name")[["Count"]].agg('sum')
puzzle1

,Count
Name,
Aadhini,6
Aadhira,34
Aadhya,468
Aadya,290
Aahana,141
...,...
Zyanya,206
Zyla,179
Zylah,143


### 如果我想要计算每年出生婴儿的总和

In [38]:
female_babynames.groupby("Year")[["Count"]].agg('sum')

,Count
Year,
1910,5950
1911,6602
1912,9804
1913,11860
1914,13815
...,...
2019,184384
2020,173966
2021,174190


使用groupby之后直接.sum()使用函数也可以

In [41]:
puzzle2 = female_babynames.groupby("Year")[["Count"]].sum()
puzzle2

,Count
Year,
1910,5950
1911,6602
1912,9804
1913,11860
1914,13815
...,...
2019,184384
2020,173966
2021,174190


### 我想按党派筛选出选票最多的总统

下面这样做的话, 只有选票是最多的, 但是year, candidate, result都和选票最多的那个候选人的信息对应不上

In [4]:
elections = pd.read_csv("../data/elections.csv")
table = elections.groupby("Party").agg('max')
table

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001
American Independent,1976,Lester Maddox,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Michael Peroutka,203091,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2020,Woodrow Wilson,81268924,win,61.344703


更好的方法:

In [57]:
table2 = elections.sort_values("%", ascending = False)
table2.head(10)

,Year,Candidate,Party,Popular vote,Result,%
114,1964,Lyndon Johnson,Democratic,43127041,win,61.344703
91,1936,Franklin Roosevelt,Democratic,27752648,win,60.978107
120,1972,Richard Nixon,Republican,47168710,win,60.907806
79,1920,Warren Harding,Republican,16144093,win,60.574501
133,1984,Ronald Reagan,Republican,54455472,win,59.023326
84,1928,Herbert Hoover,Republican,21427123,win,58.368524
86,1932,Franklin Roosevelt,Democratic,22821277,win,57.672125
109,1956,Dwight Eisenhower,Republican,35579180,win,57.650654
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
60,1904,Theodore Roosevelt,Republican,7630557,win,56.562787


有非常多种方法可以实现目标

In [60]:
table_result = table2.groupby("Party").agg(lambda x:x.iloc[0])
# table_result = table2.groupby("Party").first() 也可以
# table_result = table2.drop_duplicates(["Party"]) 也可以
table_result

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2008,Chuck Baldwin,199750,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,1964,Lyndon Johnson,43127041,win,61.344703


In [ ]:
table2.drop_duplicates(["Party"])

### groupby.size()

In [69]:
elections.groupby("Party").size().sort_values(ascending = False).head(5)

Party
Democratic     47
Republican     41
Libertarian    12
Prohibition    11
Socialist      10
dtype: int64

### groupby.filter()

In [70]:
elections.groupby("Year").filter(lambda sf:sf["%"].max() < 45)

,Year,Candidate,Party,Popular vote,Result,%
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408
24,1860,John Bell,Constitutional Union,590901,loss,12.639283
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
69,1912,William Taft,Republican,3486242,loss,23.218466
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
115,1968,George Wallace,American Independent,9901118,loss,13.571218


### 算出每一年出生的男孩女孩分别的总数

In [14]:
print(babynames.dtypes)

State    object
Sex      object
Year      int64
Name     object
Count     int64
dtype: object


,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,NaN
1,CA,F,1910,Helen,NaN
2,CA,F,1910,Dorothy,NaN
3,CA,F,1910,Margaret,NaN
4,CA,F,1910,Frances,NaN
...,...,...,...,...,...
413889,CA,M,2023,Ziah,NaN
413890,CA,M,2023,Ziaire,NaN
413891,CA,M,2023,Zidane,NaN
413892,CA,M,2023,Zyan,NaN


#### group a DataFrame by specified feature with specified columns

In [21]:
babynames.groupby("Year")["Count"].sum()

Year
1910      9163
1911      9983
1912     17946
1913     22094
1914     26926
         ...  
2019    387325
2020    363307
2021    363206
2022    361960
2023    342550
Name: Count, Length: 114, dtype: int64

#### group a DataFrame by multiple features with specified columns

In [23]:
babynames.groupby(["Year","Sex"])["Count"].agg('sum').head(6)

Year  Sex
1910  F      5950
      M      3213
1911  F      6602
      M      3381
1912  F      9804
      M      8142
Name: Count, dtype: int64

In [24]:
babynames.groupby(["Sex","Year"])["Count"].agg('sum').head(6)

Sex  Year
F    1910     5950
     1911     6602
     1912     9804
     1913    11860
     1914    13815
     1915    18643
Name: Count, dtype: int64

## 汇总表 pivot tables

In [28]:
babynames_pivot = babynames.pivot_table(
    index = "Year",
    columns = "Sex",
    values = ['Count'],
    aggfunc = 'sum',
)
babynames_pivot

Count        
Sex        F       M
Year                
1910    5950    3213
1911    6602    3381
1912    9804    8142
1913   11860   10234
1914   13815   13111
...      ...     ...
2019  184384  202941
2020  173966  189341
2021  174190  189016
2022  173353  188607
2023  163845  178705

[114 rows x 2 columns]

## 合并表格

In [31]:
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


In [34]:
male_2020_babynames = babynames.query('Sex == "M" and Year == 2020')
male_2020_babynames

,State,Sex,Year,Name,Count
402534,CA,M,2020,Noah,2631
402535,CA,M,2020,Liam,2431
402536,CA,M,2020,Mateo,2080
402537,CA,M,2020,Sebastian,1996
402538,CA,M,2020,Julian,1688
...,...,...,...,...,...
405326,CA,M,2020,Zevi,5
405327,CA,M,2020,Ziaan,5
405328,CA,M,2020,Ziad,5
405329,CA,M,2020,Ziaire,5


In [36]:
elections["First Name"] = elections["Candidate"].str.split().str[0]
elections

,Year,Candidate,Party,Popular vote,Result,%,First Name
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew
...,...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699,Jill
178,2020,Joseph Biden,Democratic,81268924,win,51.311515,Joseph
179,2020,Donald Trump,Republican,74216154,loss,46.858542,Donald
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979,Jo


 merge 函数会根据 left_on 和 right_on 指定的列中的值，在两个 DataFrame 中查找匹配项。只有当两个 DataFrame 中的键值相匹配时，相应的行才会在结果 DataFrame 中一起出现。

In [37]:
merged = pd.merge(left = elections, right = male_2020_babynames,
                 left_on = "First Name", right_on = "Name")
merged

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122,Andrew,CA,M,2020,Andrew,875
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878,John,CA,M,2020,John,623
2,1828,Andrew Jackson,Democratic,642806,win,56.203927,Andrew,CA,M,2020,Andrew,875
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073,John,CA,M,2020,John,623
4,1832,Andrew Jackson,Democratic,702735,win,54.574789,Andrew,CA,M,2020,Andrew,875
...,...,...,...,...,...,...,...,...,...,...,...,...
142,2016,Evan McMullin,Independent,732273,loss,0.539546,Evan,CA,M,2020,Evan,598
143,2016,Gary Johnson,Libertarian,4489235,loss,3.307714,Gary,CA,M,2020,Gary,22
144,2020,Joseph Biden,Democratic,81268924,win,51.311515,Joseph,CA,M,2020,Joseph,1003
145,2020,Donald Trump,Republican,74216154,loss,46.858542,Donald,CA,M,2020,Donald,40


In [38]:
merged.sort_values("Count")

,Year_x,Candidate,Party,Popular vote,Result,%,First Name,State,Sex,Year_y,Name,Count
127,1996,Ross Perot,Reform,8085294,loss,8.408844,Ross,CA,M,2020,Ross,5
122,1996,Bill Clinton,Democratic,47400125,win,49.296938,Bill,CA,M,2020,Bill,5
121,1992,Ross Perot,Independent,19743821,loss,18.956298,Ross,CA,M,2020,Ross,5
118,1992,Bill Clinton,Democratic,44909806,win,43.118485,Bill,CA,M,2020,Bill,5
96,1968,Hubert Humphrey,Democratic,31271839,loss,42.863537,Hubert,CA,M,2020,Hubert,5
...,...,...,...,...,...,...,...,...,...,...,...,...
111,1984,David Bergland,Libertarian,228111,loss,0.247245,David,CA,M,2020,David,1165
131,2004,David Cobb,Green,119859,loss,0.098088,David,CA,M,2020,David,1165
32,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838,Benjamin,CA,M,2020,Benjamin,1654
35,1888,Benjamin Harrison,Republican,5443633,win,47.858041,Benjamin,CA,M,2020,Benjamin,1654


0        151271
1        113142
2        642806
3        500897
4        702735
         ...   
177     1457226
178    81268924
179    74216154
180     1865724
181      405035
Name: Popular vote, Length: 182, dtype: int64